In [4]:
import pandas as pd


def create_data():
    # 生成示例数据
    data = {
        "x": [
            "r",
            "g",
            "r",
            "b",
            "g",
            "g",
            "r",
            "r",
            "b",
            "g",
            "g",
            "r",
            "b",
            "b",
            "g",
        ],
        "y": [
            "m",
            "s",
            "l",
            "s",
            "m",
            "s",
            "m",
            "s",
            "m",
            "l",
            "l",
            "s",
            "m",
            "m",
            "l",
        ],
        "labels": [
            "A",
            "A",
            "A",
            "A",
            "A",
            "A",
            "A",
            "A",
            "B",
            "B",
            "B",
            "B",
            "B",
            "B",
            "B",
        ],
    }
    data = pd.DataFrame(data, columns=["labels", "x", "y"])
    return data

In [5]:
data = create_data()
data

,labels,x,y
0,A,r,m
1,A,g,s
2,A,r,l
3,A,b,s
4,A,g,m
5,A,g,s
6,A,r,m
7,A,r,s
8,B,b,m
9,B,g,l


In [6]:
# 利用先验概率进行 P_labels 概率预设
# 直接从数据集里计算label相对总数据的概率，作为先验概率
def get_P_labels(labels):
    # P(\text{种类}) 先验概率计算
    labels = list(labels)  # 转换为 list 类型
    P_label = {}  # 设置空字典用于存入 label 的概率
    for label in labels:
        # 统计 label 标签在标签集中出现的次数再除以总长度
        P_label[label] = labels.count(label) / float(
            len(labels)
        )  # p = count(y) / count(Y)
    return P_label


P_labels = get_P_labels(data["labels"])
P_labels

{'A': 0.5333333333333333, 'B': 0.4666666666666667}

In [7]:
import numpy as np

# 将 data 中的属性切割出来，即 x 和 y 属性
# 取左右行，但是列从第一个开始取
train_data = np.array(data.iloc[:, 1:])
train_data

array([['r', 'm'],
       ['g', 's'],
       ['r', 'l'],
       ['b', 's'],
       ['g', 'm'],
       ['g', 's'],
       ['r', 'm'],
       ['r', 's'],
       ['b', 'm'],
       ['g', 'l'],
       ['g', 'l'],
       ['r', 's'],
       ['b', 'm'],
       ['b', 'm'],
       ['g', 'l']], dtype=object)

In [9]:
labels = data["labels"]
label_index = []
# 遍历所有的标签，这里就是将标签为 A 和 B 的数据集分开，label_index 中存的是该数据的下标
for y in P_labels.keys():
    temp_index = []
    # enumerate 函数返回 Series 类型数的索引和值，其中 i 为索引，label 为值
    for i, label in enumerate(labels):
        if label == y:
            temp_index.append(i)
        else:
            pass
    label_index.append(temp_index)
label_index  # 计算在数据中，每种label的索引

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14]]

In [10]:
# 遍历 train_data 中的第一列数据，提取出里面内容为r的数据
# rgb随便取一个？
x_index = [
    i for i, feature in enumerate(train_data[:, 0]) if feature == "r"
]  # 效果等同于求类别索引中 for 循环

x_index

[0, 2, 6, 7, 11]

In [11]:
# 取集合 x_index （x 属性为 r 的数据集合）与集合 label_index[0]（标签为 A 的数据集合）的交集
x_label = set(x_index) & set(label_index[0])
print("既符合 x = r 又是 A 类别的索引值：", x_label)
x_label_count = len(x_label)
# 这里就是用类别 A 中的属性 x 为 r 的数据个数除以类别 A 的总个数
print(
    "先验概率 P(r|A):", x_label_count / float(len(label_index[0]))
)  # 先验概率的计算公式

既符合 x = r 又是 A 类别的索引值： {0, 2, 6, 7}
先验概率 P(r|A): 0.5


In [21]:
# 计算先验概率
# 既符合 x = xi 又是 y 类别的索引值，即 P(特征 | 种类)
# p(x=xi,y) / p(y) 即 找出既是x = r 又是 y = A 的数据，再除以 y = A 的数据


def get_P_fea_lab(P_label, features, data):
    # P(\text{特征}∣种类) 先验概率计算
    # 该函数就是求 种类为 P_label 条件下特征为 features 的概率
    P_fea_lab = {}
    train_data = data.iloc[:, 1:]
    train_data = np.array(train_data)
    labels = data["labels"]
    print(f"P_label P(A)、P(B)的先验概率: {P_label}")
    print(f"P_label.keys(): { P_label.keys()}")
    print()
    # 遍历所有的标签
    for each_label in P_label.keys():
        # 上面代码的另一种写法，这里就是将标签为 A 和 B 的数据集分开，label_index 中存的是该数据的下标
        label_index = [i for i, label in enumerate(labels) if label == each_label]

        print(f"label_index P(each_label)的数量： {label_index}")

        # 遍历该属性下的所有取值
        # 求出每种标签下，该属性取每种值的概率
        for j in range(len(features)):
            # 筛选出该属性下属性值为 features[j] 的数据
            feature_index = [
                i
                for i, feature in enumerate(train_data[:, j])
                if feature == features[j]
            ]

            print(f"feature_index {features[j]}： {feature_index}")

            # set(x_index)&set(y_index) 取交集，得到标签值为 each_label,属性值为 features[j] 的数据集合
            # 例：既是x = r 又是 y = A 的数据量
            fea_lab_count = len(set(feature_index) & set(label_index))
            key = str(features[j]) + "|" + str(each_label)  # 拼接字符串

            # 计算先验概率
            # 计算 labels 为 each_label下，featurs 为 features[j] 的概率值
            # 既是x = r 又是 y = A 的数据量 /
            P_fea_lab[key] = fea_lab_count / float(len(label_index))
        print()
    return P_fea_lab


features = ["r", "m"]
# 得出的是，当 y = yi 的时候，x = xi的概率，即 P(特征 | 种类)
get_P_fea_lab(P_labels, features, data)

P_label P(A)、P(B)的先验概率: {'A': 0.5333333333333333, 'B': 0.4666666666666667}
P_label.keys(): dict_keys(['A', 'B'])

label_index P(each_label)的数量： [0, 1, 2, 3, 4, 5, 6, 7]
feature_index r： [0, 2, 6, 7, 11]
feature_index m： [0, 4, 6, 8, 12, 13]

label_index P(each_label)的数量： [8, 9, 10, 11, 12, 13, 14]
feature_index r： [0, 2, 6, 7, 11]
feature_index m： [0, 4, 6, 8, 12, 13]



{'r|A': 0.5,
 'm|A': 0.375,
 'r|B': 0.14285714285714285,
 'm|B': 0.42857142857142855}

In [28]:
# 1.求出所有 label 的先验概率,极大似然估计  P(A)、P(B) 
# 2.求出每个 feature 的先验概率，极大似然估计  P(r | A)、P(r | B)、P(m | A)、P(m | B)
# 3.计算总特征下的概率 P(A | [r,m]) 
def classify(data, features):
    # 朴素贝叶斯分类器
    # 求 labels 中每个 label 的先验概率
    labels = data["labels"]
    # 这里也就是求 P（B），P_label 为一个字典，存的是每个 B 对应的 P(B)
    P_label = get_P_labels(labels)
    P_fea_lab = get_P_fea_lab(P_label, features, data)  # 这里是在求 P（A|B）

    print(f"classify P_label:{P_label}")
    print(f"classify P_fea_lab:{P_fea_lab}")
    P = {}
    P_show = {}  # 后验概率
    for each_label in P_label:
        P[each_label] = P_label[each_label]
        # 遍历每个标签下的每种属性
        for each_feature in features:
            # 拼接字符串为 P(B/A) 用于字典的键值
            # A | [r,m]
            key = str(each_label) + "|" + str(features)

            # r | A, m | A......
            show_key = str(each_feature) + "|" + str(each_label)

            # 计算 P(B/A) = P(B) * P(A/B) 因为所有的后验概率，分母相同。因此，在计算时可以忽略掉。
            # P(A | [r,m]) = P(A) * P(r | A) * P(m | A)
            
            # 对于公式：P(B | A) = P(A | B) * P(B) / P(A)
            # A = [r,m],B = A,则有 P(A,[r,m]) = P([r,m] | A) * P(A) / P([r,m])
            # P([r,m] | A) = P(r | A) * P(m | A)
            # P([r,m])?
            # 则有 P(A,[r,m]) = P(r | A) * P(m | A) * P(A) / ?
            # 通常我们只关心哪个后验概率最大，而不需要精确的概率值，所以分母一样的情况下可以忽略，则有 P(A,[r,m]) = P(r | A) * P(m | A) * P(A)
            temp_value = P[each_label] * P_fea_lab[show_key]
            
            P_show[key] = temp_value
            # 把刚才算的概率放到 P 列表里面，这个 P 列表的键值变成了标签。
            # 这样做的目的，其实是为了在后面取最大时，取出就是标签，而不是 标签|特征
            P[each_label] = temp_value
    print()
    # 输出 P_show 和 P 观察，发现他们的概率值没有变，只是字典的 key 值变了
    print(P_show)
    print(P)
    features_label = max(P, key=P.get)  # 概率最大值对应的类别
    return features_label

In [29]:
classify(data, ["r", "m"])

P_label P(A)、P(B)的先验概率: {'A': 0.5333333333333333, 'B': 0.4666666666666667}
P_label.keys(): dict_keys(['A', 'B'])

label_index P(each_label)的数量： [0, 1, 2, 3, 4, 5, 6, 7]
feature_index r： [0, 2, 6, 7, 11]
feature_index m： [0, 4, 6, 8, 12, 13]

label_index P(each_label)的数量： [8, 9, 10, 11, 12, 13, 14]
feature_index r： [0, 2, 6, 7, 11]
feature_index m： [0, 4, 6, 8, 12, 13]

classify P_label:{'A': 0.5333333333333333, 'B': 0.4666666666666667}
classify P_fea_lab:{'r|A': 0.5, 'm|A': 0.375, 'r|B': 0.14285714285714285, 'm|B': 0.42857142857142855}

{"A|['r', 'm']": 0.1, "B|['r', 'm']": 0.02857142857142857}
{'A': 0.1, 'B': 0.02857142857142857}


'A'